Run cells bellow to download and preprocess the **Steam-200K** dataset.

Estimated time is around 45 minutes.

In [93]:
#!unzip /home/ubuntu/vasp/Datasets/steam-200k.zip

In [94]:
import pandas as pd
import os

All Items

In [95]:
ds_path = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_100"
ds_path_out = "/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp"

In [96]:
temp_tr = pd.read_csv(os.path.join(ds_path, 'train_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'train_te.csv'))
train = pd.concat([temp_tr, temp_te])
train.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(train.head())
print(len(train))

temp_tr = pd.read_csv(os.path.join(ds_path, 'validation_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'validation_te.csv'))
val = pd.concat([temp_tr, temp_te])
val.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(val.head())
print(len(val))

temp_tr = pd.read_csv(os.path.join(ds_path, 'test_tr.csv'))
temp_te = pd.read_csv(os.path.join(ds_path, 'test_te.csv'))
test = pd.concat([temp_tr, temp_te])
test.rename({'uId': 'userid', 'iId': 'itemid'}, axis=1, inplace=True)
print(test.head())
print(len(test))

full = pd.concat([train, val, test])
print(full.head())
print(len(full))

   userid  itemid                                        title
0     543     110                               Counter-Strike
1     543     532                    Call of Duty World at War
2     543     325                             Total War ATTILA
3     543     183  Call of Duty Modern Warfare 2 - Multiplayer
4     543     167                Call of Duty Modern Warfare 2
23203
   userid  itemid                       title
0    1420      91             Team Fortress 2
1    1420      10  Sid Meier's Civilization V
2    1420       0  The Elder Scrolls V Skyrim
3    1420      23                   Far Cry 3
4    1420       3           Fallout New Vegas
2999
   userid  itemid                                        title
0    1622     112                Counter-Strike Condition Zero
1    1622     183  Call of Duty Modern Warfare 2 - Multiplayer
2    1622     110                               Counter-Strike
3    1622     114                       Call of Duty Black Ops
4    1622      77   

In [97]:
items = pd.read_csv(os.path.join(ds_path, 'unique_iId.txt'))
items['iId']=items['iId'].apply(str)
items.rename({'iId': 'itemid'}, axis=1, inplace=True)

items.to_json(os.path.join(ds_path_out, 'items.json'))
items

,itemid
0,0
1,1
2,2
3,3
4,4
...,...
2114,5131
2115,2971
2116,5152
2117,5065


In [98]:
result = train['itemid'].isin(test['itemid'])

result.unique()

array([ True, False])

Purchases and grouped purchases.

In [100]:
interactions = full

interactions['itemid'] = interactions['itemid'].apply(str)
interactions['userid'] = interactions['userid'].apply(str)
interactions.reset_index(drop=True, inplace=True)
interactions.to_json(os.path.join(ds_path_out, 'purchases.json'))

interactions['itemids'] = interactions[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = interactions[['userid','itemids']].drop_duplicates()
iii.reset_index(drop=True, inplace=True)
iii.to_json(os.path.join(ds_path_out, 'purchases_txt.json'))
print(iii.head())

  userid                                            itemids
0    543  110,532,325,183,167,89,186,114,272,34,56,115,2...
1    797                            0,119,23,4,2086,218,276
2    591                             89,88,243,167,4,49,543
3    105  270,115,505,408,183,888,898,34,548,1931,4,232,...
4    953                        88,10,48,110,81,57,1597,246


Keep only users with 5 or more interactions.

Purchases.

In [101]:
purchases=pd.read_json(os.path.join(ds_path_out, 'purchases.json'))
purchases['userid'] = purchases.userid.apply(str)
purchases['itemid'] = purchases.itemid.apply(str)
#purchases_item_counts = purchases[['uId','iId']]
#purchases_user_counts = purchases[['uId','iId']]
purchases_user_count = purchases.groupby(['userid']).size().to_frame('nr_of_purchases').reset_index()
purchases_user_count = purchases_user_count.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_user_count))
pu5=purchases_user_count#[purchases_user_count.nr_of_purchases>=5]
print(len(purchases_user_count))
print(len(pu5))

purchases_pu5 = purchases[purchases.userid.isin(pu5.userid)]
purchases_item_count_pu5 = purchases_pu5.groupby(['itemid']).size().to_frame('nr_of_purchases').reset_index()
purchases_item_count_pu5 = purchases_item_count_pu5.sort_values(by=['nr_of_purchases'], ascending=False)
print(len(purchases_item_count_pu5))

#purchases_pu5.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))
purchases.to_json(os.path.join(ds_path_out, 'purchases_pu5.json'))

1656
1656
1656
2119


Grouped purchases.

In [102]:
purchases_pu5['itemids'] = purchases_pu5[['userid','itemid']].groupby(['userid'])['itemid'].transform(lambda x: ','.join(x))
iii = purchases_pu5[['userid','itemids']].drop_duplicates()
iii['uId']=iii['userid'].apply(str)
iii = iii[['userid','itemids']]
iii.to_json(os.path.join(ds_path_out, 'purchases_txt_pu5.json'))

Users.

In [103]:
iii['userid'].to_frame().to_json(os.path.join(ds_path_out, 'users_pu5.json'))
print(iii['userid'].to_frame().head())

   userid
0     543
12    797
18    591
24    105
60    953


Items

In [104]:
#items[items.itemid.isin(purchases_item_count_pu5.itemid)].to_json("items_pu5.json")
items[items.itemid.isin(items.itemid)].to_json(os.path.join(ds_path_out, 'items_pu5.json'))

Items sorted by number of interactions

In [105]:
purchases_item_count_pu5.to_json(os.path.join(ds_path_out, 'items_sorted_pu5.json'))
purchases_item_count_pu5

,itemid,nr_of_purchases
1986,88,679
2020,91,616
1975,87,462
1453,4,456
0,0,449
...,...,...
965,1867,1
970,1871,1
971,1872,1
972,1873,1


Users sorted by number of interactions

In [106]:
pu5.to_json(os.path.join(ds_path_out, 'users_sorted_pu5.json'))
pu5

,userid,nr_of_purchases
347,131,216
281,1250,166
455,1407,141
763,195,140
1245,629,135
...,...,...
1335,71,5
540,1484,5
1332,707,5
544,1489,5


Create train, val and test split.

In [107]:
#users = pd.read_json('users_pu5.json')
#shuffled_users = users.sample(frac=1., random_state=42)
test_users = test['userid'].to_frame().drop_duplicates(subset=['userid'])
val_users = val['userid'].to_frame().drop_duplicates(subset=['userid'])
train_users = train['userid'].to_frame().drop_duplicates(subset=['userid'])

test_users.to_json(os.path.join(ds_path_out, "test_users.json"))
print(test_users.head())
val_users.to_json(os.path.join(ds_path_out, "val_users.json"))
print(val_users.head())
train_users.to_json(os.path.join(ds_path_out, "train_users.json"))
print(train_users.head())

len(train_users),len(val_users),len(test_users)

    userid
0     1622
6     1649
15    1637
24    1497
36    1514
     userid
0      1420
59     1399
118    1395
138    1369
146    1397
    userid
0      543
12     797
18     591
24     105
60     953


(1328, 165, 163)

List of generated json files with preprocessed data.

In [108]:
!ls /home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/*.json

/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/items.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/items_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/items_sorted_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/purchases.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/purchases_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/purchases_txt.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/purchases_txt_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/test_users.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/train_users.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/users.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/users_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/users_sorted_pu5.json
/home/ubuntu/vasp/Datasets/steam-200k/preprocessed_vasp/val_users.json
